In [11]:
import os
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [12]:
create_dir("D:\projects\segmentation of aerial imagery\data\org_data\image")
create_dir("D:\projects\segmentation of aerial imagery\data\org_data\mask")